# Хакатон ООО "Просепт" и Яндекс Мастерская 23 ноября - 14 декабря 2023

**Мэтчинг товаров диллеров с товарами заказчика**

**Описание задачи**
Заказчик производит несколько сотен различных товаров бытовой и промышленной химии, а затем продаёт эти товары через дилеров. Дилеры, в свою очередь, занимаются розничной продажей товаров в крупных сетях магазинов и на онлайн площадках.

Для оценки ситуации, управления ценами и бизнесом в целом, заказчик периодически собирает информацию о том, как дилеры продают их товар. Для этого они парсят сайты дилеров, а затем сопоставляют товары и цены. Зачастую описание товаров на сайтах дилеров отличаются от того описания, что даёт заказчик. Например, могут добавляться новый слова (“универсальный”, “эффективный”), объём (0.6 л -> 600 мл). Поэтому сопоставление товаров дилеров с товарами производителя делается вручную.

**Цель этого проекта** - разработка решения, которое отчасти автоматизирует процесс сопоставления товаров. Основная идея - предлагать несколько товаров заказчика, которые с наибольшей вероятностью соответствуют размечаемому товару дилера. Предлагается реализовать это решение, как онлайн сервис, открываемый в веб-браузере. Выбор наиболее вероятных подсказок делается методами машинного обучения.

**Цель работы DS в этом проекте** - разработка алгоритма подбора наивероятнейших соответствий и связка с beckend частью/

**Над проектом трудились:**
1. Минайчев Денис https://github.com/SeriousDen7 (сборка проекта, оборачивание в функции, очистка названий, выделение фичей из названий, методика поэтапного сбора предсказаний: артикул, TF-IDF, МО классификация)
2. Носач Ирина https://github.com/LolaBSL (очистка названий, лемматизация, проработка машинного обучения после перевода в задачу классификации)
3. Мукосеев Андрей(до 28 ноября) https://github.com/Andrey-Mukoseev (Проверка языковых моделей (BERT, TF-IDF),  исследование задачи классификации после ранжирования)

**Основные этапы выполнения проекта**

1. Изучение данных.
2. Генерация гипотез, их проверка
3. Подготовка данных
4. Обучение модели и оценка метрики
5. Упаковка модели, написание py-скриптов

**Описание данных**
Заказчик предоставил несколько таблиц (дамп БД), содержащих необходимые
данные:
1. marketing_dealer - список дилеров;
2. marketing_dealerprice - результат работы парсера площадок дилеров:
    1.  product_key - уникальный номер позиции;
    2. price - цена;
    3. product_url - адрес страницы, откуда собраны данные;
    4. product_name - заголовок продаваемого товара;
    5. date - дата получения информации;
    6. dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)
3. marketing_product - список товаров, которые производит и распространяет заказчик;
    1. article - артикул товара;
    2. ean_13 - код товара (см. EAN 13)
    3. name - название товара;
    4. cost - стоимость;
    5. min_recommended_price - рекомендованная минимальная цена;
    6. recommended_price - рекомендованная цена;
    7. category_id - категория товара;
    8. ozon_name - названиет товара на Озоне;
    9. name_1c - название товара в 1C;
    10. wb_name - название товара на Wildberries;
    11. ozon_article - описание для Озон;
    12. wb_article - артикул для Wildberries;
    13. ym_article - артикул для Яндекс.Маркета;

4. marketing_productdealerkey - таблица матчинга товаров заказчика и товаров дилеров
    1. key - внешний ключ к marketing_dealerprice
    2. product_id - внешний ключ к marketing_product
    3. dealer_id - внешний ключ к marketing_dealer

**Цель исследования:** 
1. Создание модели машинного обучения классификации комментарии на токсичные и не токсичные.

**Задачи исследования:**
1. Предобработать данные
2. Лемматизация, эмбеддинг и пр.
3. Разделить данные на выборки (обучающая и тестовая, признаки и целевой показатель)
4. Проверить модели машинного обучения на данных
5. Сравненить результаты моделей по метрике F1.

**Объектом исследования** являются данные 3 таблиц: парсерные данные, продукция производителя, размеченые соответствия персерных данных продукции.

**Предметом исследования** является комплекс методик машинного обучения и инструментов связанных с МО и обработки текстов.

**Выводы по работе:**
1) Из названий можно вытащить большое количество данных, которые можно рассматривать отдельно от текста(артикул, единицы измерения, количество/объем)
2) В таблице продукции производителя 4 колонки с названиями, и можно взять все уникальные слова из этих колонок
3) Зачистка большого количества мусора в названиях и некорректные названия(слипшиеся слова) увеличивает метрику
4) Обучать по размеченным данным может не требоваться, т.к. в предварительном варианте работы идет просчет расстояния между векторами, и только по нему ранжируются предсказания
5) Предполагается доработка: перевод в задачу классификации и использование размеченных данных как учителя
6) Полученные в процессе обработки названий дополнительные факторы(объем и единица измерения) вместе с ценой, которая дана, не были использованы в данном решении, но могут использоваться при реализации п.5
7) Все процессы выполняются в рамках функций и объединяются в одной функции, которая будет использована в beckend'е
8) Показатели метрики: в 83% случаев правильный ответ находится в топ-5 предсказаний


**Использованные приемы и инструменты:** Регулярные выражения, лемматизация, TF-IDF, faiss


Оставшиеся вопросы и не проверенные гипотезы:
1) Пайплайн или нет? Возможно
2) Покажет ли BERT лучший результат на очишенных данных (на не очищенных данных был неудовлетворительный результат)
3) Градиентный бустинг в какой момент? В момент реализации задачи классификации
2) Мешок слов
3) N-граммы

In [1]:
!pip install pandas
!pip install numpy
!pip install spacy
!pip install faiss-cpu
!pip install nltk
!pip install scikit-learn

In [ ]:
!python -m spacy download ru_core_news_sm
!python -m spacy download en_core_web_sm

In [1]:
import re
import numpy as np
import pandas as pd
import spacy
import nltk
import faiss

from collections import OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from pickle import dump, load

In [19]:
def csv_reading(path=''):
    '''
    Функция загрузки данных. по умолчани берет файлы из директории 
    Выдает 4 датафрейма (возможно следует таблицу мэтчей сделать опциональным аргументом, без МО(с учителем) она не нужна)
    '''
    dealers = (pd.read_csv(path+'marketing_dealer.csv', sep=';').sort_values('id')
                                                               .reset_index(drop=True)) 
    parser = pd.read_csv(path+'marketing_dealerprice.csv', sep=';')
    products = pd.read_csv(path+'marketing_product.csv', sep=';')
    matches = pd.read_csv(path+'marketing_productdealerkey.csv', sep=';')
    return dealers, parser, products, matches

**Функции**

In [3]:
def text_worker(name): 
    '''
    Функция для предварительной очистки текста
    Принимает строку, возвращает строку
    
    Добавляет пробел на явных стыках, переходах(язык, регистр) 
    и на конкретных словах
    Переводит все в нижний регистр
    Убирает название заказчика
    '''
    pattern = [r"([а-я])([a-zA-ZА-Я])", 
               r"([А-Я])([A-Za-z])", 
               r"([a-z])([A-Zа-яА-Я])"]     
    for p in pattern:
        try:
            name = re.sub(p, "\\1 \\2", name)
        except:
            pass
    name = name.lower()
    bad = ['просепт','prosept50',
           'prosept50,', 'prosepteco50',
           'prosepteco50,','ultra', 
           'crystal', '-ая', 
           'prosept','ф/п']
    good = ['prosept',' prosept50 ',
            ' prosept50 ', ' prosepteco50 ', 
            ' prosepteco50 ', ' ultra ', 
            ' crystal ', '', 
            '', 'пакет']
    for o,n in zip(bad, good):
        name = name.replace(o,n)
    return name

In [4]:
def change_equal(name): 
    '''
    Функция для разделения названия на текст и другие признаки
    
    Принимает текст, возвращает словарь вида 
    {признак: его значение}
    Среди обнаруженных признаков:
    1)Название очищеное от следующих признаков
    2)Артитул (может выстапуть для 100% нахождения соответствующего ID продукта)
    3)Количество/объем цифрой  
    4)Еденицы измерения 
    Последние 2 признака переводит в общий формат (1000 г->1 кг, 600 мл->0,6 л) 
    '''
    name = text_worker(name)
    dictionary={}
    try:
        article = re.search(r' \d+-\d+/?\d?[а-я]?',name)[0].strip()
        name = name.replace(article,'')
    except:
        article = np.nan
    try:
        v = re.findall(r'\d+(?:[\.,]\d+)? ?(?:мл|кг|г|л|шт)',name)[-1]
        dimension = re.search(r'[млкгшт]+',v)[0]
        quantity = float(v.replace(dimension,'').replace(',','.'))
        if dimension == 'мл':
            quantity = quantity/1000
            dimension = 'л'
        elif dimension == 'г':
            quantity = quantity/1000
            dimension = 'кг'
        name = name.replace(v,'')
    except:
        dimension = np.nan
        quantity = 0
    finally:
        dictionary['name_new'] = ' '.join(re.sub(r'\W+', ' ', name).split(r'\W+'))
        dictionary['article'] = article
        dictionary['dimension'] = dimension
        dictionary['quantity'] = quantity
        return dictionary        

In [5]:
def lemmatizate(text, nlp, nlp_ru):
    '''
    Функция лемматизации текста, 
    принимает строку и выдает так же строку но все слова в унифицированной форме 
    '''
    
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    doc_ru = nlp_ru(text)
    return " ".join([token.lemma_ for token in doc_ru])

In [6]:
def parser_prep(parser):
    '''
    Функция обработки таблицы dealer_price(парсерные данные)
    Применение функции по обработке текста, разделения названия на признаки, лемматизации
    '''
    # Удаляем 3 ненужные колонки, удаляем дубликаты, переименовываем колонку ключа, и выставляем новые индексы
    parser = (parser
              .drop(['id','product_url', 'date'], axis=1)
              .drop_duplicates()
              .rename(columns={'product_key': 'key'})
              .reset_index(drop=True))
    parser_new = (parser['product_name']
                  .apply(lambda x: change_equal(x)).apply(pd.Series))
    parser_final = (pd.concat([parser, parser_new],axis=1)
                    .drop(['product_name'], axis=1))
    parser_final['article'] = parser_final['article'].fillna('')
    
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    nlp_ru = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])
    parser_final['name_new'] = (parser_final['name_new']
                                .apply(lambda x: lemmatizate(x,nlp, nlp_ru)))
    
    return parser_final

In [7]:
def products_prep(products): 
    '''
    Функция обработки таблицы продукции
    Принимает исходную таблицу с проведенной предподготовкой
    Выдает винальный вариант таблицы, который можно использовать в МО
    
    Заполняет пропуски, собирает в одну колонку уникальные слова из 4 колонок названий, и собирает дополнительные фичи из названий
    '''
    # Удаляем 7 ненужных колонок
    products = (products
                .drop(['Unnamed: 0', 'ean_13',
                      'category_id', 'ozon_article', 
                      'wb_article', 'ym_article', 
                      'wb_article_td'], axis=1))
    #заполняем пропуски
    name_columns = ['name_1c', 'ozon_name', 'name', 'wb_name']
    products['name_1c'] = products['name_1c'].fillna(products['name'])
    products['ozon_name'] = products['ozon_name'].fillna(products['name_1c'])
    products[name_columns] = products[name_columns].fillna('')
    
    #Из субъективно лучшей колонки названия вытаскиваем количество и ед.измерения из остальных только очищеное имя
    name_1c = (products['name_1c']
               .apply(change_equal).apply(pd.Series)
               .rename(columns={'name_new': 'name_1c_new'})['name_1c_new'])
    ozon_name = (products['ozon_name']
                 .apply(change_equal).apply(pd.Series)
                 .drop(['article'],axis=1)
                 .rename(columns={'name_new': 'ozon_name_new'}))
    name_1 = (products['name']
              .apply(change_equal).apply(pd.Series)
              .rename(columns={'name_new': 'name_new'})['name_new'])
    wb_name = (products['wb_name']
               .apply(change_equal).apply(pd.Series)
               .rename(columns={'name_new': 'wb_name_new'})['wb_name_new'])
    products1 = pd.concat([products, ozon_name, name_1c, wb_name, name_1],axis=1)
    
    #Соединяем названия из 4х колонок в full name (только уникальные слова в том же порядке)
    products1['full_name'] = (products1
                              .apply(lambda x: ' '.join(list(dict.fromkeys((x['name_1c_new']+' '
                                                                            +x['ozon_name_new']+' '
                                                                            +x['name_new']+' '
                                                                            +x['wb_name_new']).split()))), axis=1))
    
    #лемматизируем
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    nlp_ru = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])
    products1['full_name'] = (products1['full_name']
                              .apply(lambda x: lemmatizate(x,nlp, nlp_ru)))
    
    products1['article'] = products1['article'].apply(lambda x: x.lower())
    
    #Собираем финальную версию таблицы продукции
    products_final = products1.loc[:,['id','article',
                                      'cost','recommended_price', 
                                      'full_name', 'dimension', 
                                      'quantity']]
    
    #заполняем пропуски рекомендованной цены медианой
    products_final['recommended_price'] = (products_final['recommended_price']
                                           .fillna(products_final['recommended_price'].median()))
    return products_final

In [8]:
def find_product_id_by_article(article, products_final):
    '''
    Фанкция нахождения id товара по артикулу
    
    Принимает на вход артикул строкой, 
    возвращает id товара из таблицы продукции списком из 1 числа
    '''
    id = []
    if article == '':
        return np.nan
    else:
        #print(f'Артикул {article}, будет поиск')
        try:
            id.append(int(list(
                products_final.loc[products_final['article']==article].id)[0]))
        except:
            return np.nan
        return id

In [9]:
def accuracy_check(target, dict_of_preds,n):
    '''
    Функция промежуточной проверки точности
    Принимает правильные соответствия в виде словаря, словарь предсказаний и рассматриваемое количество первых предсказаний
    Выводит текст с оценкой, в переменную никакую не записывается
    '''
    
    acc = 0
    n_empty = 0
    for key in list(target.keys()):
        n_empty +=int((key not in dict_of_preds) 
                      or (dict_of_preds[key]==[]))
        acc += int((key in dict_of_preds) 
                   and (target[key] in dict_of_preds[key][:n]))
    try:
        result = round(100 * acc / (len(list(target.keys()))-n_empty),2)
        return (f'В {result}% случаев правильный ответ среди первых {n} предсказаний.'
                f' Количество сравнений: {(len(list(target.keys()))-n_empty)}')
    except ZeroDivisionError:
        return 'Метрика не подсчитана, деление на 0, необходимо проверить условие в функции "accuracy_check"'

In [10]:
def fit_tf_idf(products_final,parser_final):
    '''
    Функция для обучения векторайзера текста
    
    Принимает 2 таблицы, составляет корпус для обучения,
    возвращает обученный векторайзер TF-IDF
    '''
    try:
        with open('korpus_tfidf.pkl', 'rb') as fid:
            count_tf_idf = load(fid)
        print("Зарузили векторайзер из файла")
    except:
        korpus = pd.concat([products_final['full_name'], 
                            parser_final['name_new']])
        nltk.download('stopwords')
        stopwords=set(nltk_stopwords.words('russian'))
        count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))
        count_tf_idf.fit(korpus)
        with open('korpus_tfidf.pkl', 'wb') as fid:
            dump(count_tf_idf, fid)
        print('Обучили векторайзер и сохранили')
        
    return count_tf_idf

In [11]:
def search_neighbors(base_tfidf, query_tfidf):
    '''
    Функция ранжирования и отбора ближайших соседей 
    по расстоянию между векторизированными текстами
    
    Принимает на вход 2 таблицы векторизированного текста
    выдает (base_index) словарь соответствия индекса с id продукта,
    (vecs) список списков расстояний между векторами по возрастанию (n ближайших)
    (idx) список списков ближайших n id продукции в соответствии с (vecs)
    
    '''
    dims = base_tfidf.shape[1]
    n_cells = 1 # количество центроидов
    quantizer = faiss.IndexFlatL2(dims)
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    # подготовка к поиску
    idx_l2.train(np.ascontiguousarray(base_tfidf.values).astype('float32'))
    idx_l2.add(np.ascontiguousarray(base_tfidf.values).astype('float32'))
    # создание словаря для нахождения индекса товара в базовом наборе данных
    base_index = {k: v for k, v in enumerate(base_tfidf.index.to_list())}
    vecs, idx = idx_l2.search(
        np.ascontiguousarray(query_tfidf.values).astype('float32'), 
        10)
    return base_index, vecs, idx

In [12]:
def get_neighbors(products_final, parser_final, matches=pd.DataFrame({'A' : []})):
    '''
    Функция поиска ближайших соседей 
    (включает использование функций обучения векторайзера и самого поиска соседей)
    Если передаем таблицу мэтчей - значит в работу берем только данные с известным ответом и будет показана метрика
    
    Принимает 2 обработанные таблицы(продукция и парсерные данные) и опционально таблицу мэтчей
    Выдает словарь соответствия ключа предполагаемым 10 id и при зарузке таблицы мэтчей - таргет
    '''
    #Создадим две таблицы. Первая: base - данные производителя, где индексы - это id товаров. 
    #Вторая: query - соответствие запроса к парсеру и правильного ответа к продукции
    base = products_final.copy().set_index('id').drop(['cost'],axis=1)
    if matches.empty:
        query = (parser_final
                 .groupby('key').first()
                 .reset_index()[['key','price', 
                                 'name_new', 'article',
                                 'quantity','dimension',
                                 'product_id']])
    else:
        # Удаляем 2 ненужные колонки в таблице мэтчей
        matches = matches.drop(['id','dealer_id'], axis=1)
        #Соединяем по таблице мэтчей и выделяем таргет
        parser_matches = (parser_final
                          .merge(matches, 
                                 how='right',
                                 left_on='key',
                                 right_on='key')
                          .dropna(subset=['name_new'])
                          .drop_duplicates()
                          .groupby('key').first()
                          .reset_index())
        parser_matches = parser_matches[['key','price', 
                                         'name_new', 'article',
                                         'quantity','dimension',
                                         'product_id']]
        targets = {a: b for a,b in zip(parser_matches['key'],
                                       parser_matches["product_id"])}
        query = parser_matches.drop("product_id", axis=1)
    #обучение TF-IDF функцией    
    count_tf_idf = fit_tf_idf(products_final, parser_final)
    
    base_tfidf = pd.DataFrame(count_tf_idf.transform(base['full_name'])
                              .toarray()).set_index(base.index)
    query_tfidf = pd.DataFrame(count_tf_idf.transform(query['name_new'])
                               .toarray())
    base_tfidf.columns = base_tfidf.columns.astype('str')
    query_tfidf.columns = query_tfidf.columns.astype('str')
    #Поиск соседей функцией
    base_index, vecs, idx = search_neighbors(base_tfidf, query_tfidf)
    
    #Сборка словаря предсказаний
    dict_of_neighbors={}
    for k,v in zip(query['key'], idx.tolist()):
        list_of_preds = [base_index[i] for i in v]
        dict_of_neighbors[k] = list_of_preds
    
    if matches.empty:
        return dict_of_neighbors
    else:  
        #Подсчет метрики
        print('TF-IDF:',accuracy_check(targets, dict_of_neighbors,5))
        return dict_of_neighbors, targets

In [13]:
def dict_join(dict_all, dict_small):
    '''
    Функция объединения значений двух словарей
    с учетом позиций
    
    На входе основной и дополнительный словари
    На выходе обновленные основной словарь
    '''
    for key in dict_small.keys():
        #print('Ключ',key)
        #print('Список для этого ключа в исходном словаре',dict_of_neigh[key])
        l1 = dict_all[key]
        l2 = dict_small[key]
        # в этот момент происходит слияние с учетом позиции
        l_all = list(dict.fromkeys(l1 + l2))
        #print(f'В общем словаре по ключу {key} значения: {l1}, в новом такие:{l2}, вместе: {l_all}')
        #print('Замена в этом списке:',dict_of_neigh[key])
        dict_all[key] = l_all
        #print('После замены:',dict_of_neigh[key])
    return dict_all

In [17]:
def main_function(path=''):
    '''
    Главная функция которая задействует все остальные функции,
    Принимает на вход путь к файлам, 
    выдает словарь соответствия ключа предложения диллера с 10 id продукции заказчика
    
    '''
    # загрузка документов
    dilers , parser, products, matches = csv_reading(path)
    # преобразование текста(обе таблицы, лемматизация)
    parser_final = parser_prep(parser)
    products_final = products_prep(products)
    #Составляем макет словаря
    dict_of_neigh = dict.fromkeys(parser_final['key'],[])
    
    #Поиск по артикулу
    dict_of_article_neigh = (pd.DataFrame(parser_final['article']
                                           .apply(lambda x: find_product_id_by_article(x, products_final)))
                              .set_index(parser_final['key'])
                              .dropna().to_dict()['article'])
    #Заносим найденное в основной словарь
    dict_of_neigh.update(dict_of_article_neigh)
    
    # поиск с помощью Tf-idf и faiss
    dict_of_tfidf_neighbors, targets = get_neighbors(products_final, 
                                                     parser_final, 
                                                     matches)
    if ~matches.empty:
        print('Поиск по артикулам:', accuracy_check(targets, dict_of_neigh,1)) 
    '''
    Эти этапы не реализованы, но предполагают возможное внедрение для улучшения точности
    # сборка каждый к каждому(query-base)
    
    #train-test split
    
    # классификация через модель
    
    # отбор топ-n по predict-proba
    '''
    
    # Сбрка финального словаря предсказаний
    dict_of_neighbors = dict_join(dict_of_neigh, dict_of_tfidf_neighbors)
    if ~matches.empty:
        print('Общий вывод:', accuracy_check(targets, dict_of_neighbors,5))
    return  dict_of_neighbors

In [18]:
%%time
dict_final = main_function()

Зарузили векторайзер из файла
TF-IDF: В 79.28% случаев правильный ответ среди первых 5 предсказаний. Количество сравнений: 1602
Поиск по артикулам: В 100.0% случаев правильный ответ среди первых 1 предсказаний. Количество сравнений: 324
Общий вывод: В 83.02% случаев правильный ответ среди первых 5 предсказаний. Количество сравнений: 1602
CPU times: total: 1min 21s
Wall time: 1min 19s


In [27]:
dict_final

{'546227': [12, 13, 15, 503, 504, 436, 282, 281, 4, 5],
 '546408': [45, 434, 470, 29, 28, 503, 504, 436, 26, 27],
 '546234': [18, 503, 504, 436, 477, 478, 489, 12, 13, 62],
 '651258': [403, 405, 404, 503, 504, 436, 211, 210, 209, 392],
 '546355': [39, 482, 503, 504, 436, 480, 286, 321, 322, 20],
 '831859': [397, 396, 503, 504, 436, 323, 324, 213, 212, 81],
 '546406': [41, 40, 503, 504, 436, 30, 513, 26, 27, 45],
 '831858': [397, 396, 503, 504, 436, 323, 324, 213, 212, 81],
 '857015': [503, 504, 436, 364, 362, 495, 237, 236, 223, 448],
 '651265': [106, 509, 506, 100, 508, 503, 504, 436, 97, 105],
 '546257': [201, 200, 202, 203, 503, 504, 436, 204, 205, 453],
 '831854': [274, 276, 277, 275, 273, 272, 271, 278, 249, 252],
 '831857': [399, 398, 487, 503, 504, 436, 208, 207, 17, 112],
 '382757': [273, 272, 271, 503, 504, 436, 245, 246, 247, 276],
 '546322': [123, 124, 125, 503, 504, 436, 84, 86, 85, 73],
 '831860': [401, 402, 400, 503, 504, 436, 392, 391, 390, 211],
 '832157': [249, 252, 25